# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

1. The author wants to explore a small but important part of data cleaning, data tidying. He provides specific criteria a dataset should meet in order to be considered tidy.
2. The tidy standard is intended to create a standard by which all data can be cleaned efficiently and ready to be studied.
3. The first sentence suggests that tidy datasets have standards that make them easy to research, but messy datasets are unique in their messiness. In context determining what the variables and obsersvations are but hard to set a concrete definition for both of them.
4. Values are either quantitative or qualitative and every value belongs to a variable and observation. Variables contain all values that measure the same attribute across units. Observations contain all values that measured on the same unit across attributes.
5. Each variable froms a column, each observation forms a row and each type of observational unit forms a table.
6. The five most common problems in messy datasets are: Column headers are values not variable names, multiple variables are stored in one column, variables are store in both rows and columns, multiple types of observational units are store in the same table, a single observational unit is stored in multiple tables. Column headers are values not variables. Melting a dataset is the process of turning column-value variables into rows.
7. Table 11 has days along the top, which are are values, table 12 melts those days into a specific variable date.
8. Wickham wants to create a more robust ecosystem of ideas and tools for data science.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('./data/airbnb_hw.csv', low_memory=False)
print( df.shape, '\n')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/airbnb_hw.csv'

In [ ]:
price = df['Price']
price.unique()

In [ ]:
price = df['Price']
price = price.str.replace(',','') # Replace commas with nothing
print( price.unique() , '\n')
price = pd.to_numeric(price,errors='coerce') # Typecast price to float/numeric
print( price.unique() , '\n')
print( 'Total missing: ', sum( price.isnull() ) )

In [3]:
df = pd.read_csv('./data/sharks.csv', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: './data/sharks.csv'

In [ ]:
df['Type'].value_counts()

In [ ]:
type = df['Type'] # Create a temporary vector of values for the Type variable to play with

type = type.replace(['Sea Disaster', 'Boat','Boating','Boatomg'],'Watercraft') # All watercraft/boating values
type.value_counts()

type = type.replace(['Invalid', 'Questionable','Unconfirmed','Unverified','Under investigation'],np.nan) # All unclean values
type.value_counts()

df['Type'] = type # Replace the 'Type' variable with the cleaned version
del type # Destroy the temporary vector

df['Type'].value_counts()

In [ ]:
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].replace(['UNKNOWN', 'F','M','2017'],np.nan) # All unclean values
df['Fatal (Y/N)'] = df['Fatal (Y/N)'].replace('y','Y') # All unclean values
pd.crosstab(df['Type'],df['Fatal (Y/N)'],normalize='index')

In [ ]:
df = pd.read_csv('./data/VirginiaPretrialData2017.csv', low_memory=False)

In [ ]:
release = df['WhetherDefendantWasReleasedPretrial']
print(release.unique(),'\n')
print(release.value_counts(),'\n')
release = release.replace(9,np.nan) # In the codebook, the 9's are "unclear"
print(release.value_counts(),'\n')
sum(release.isnull()) # 31 missing values
df['WhetherDefendantWasReleasedPretrial'] = release # Replace data column with cleaned values
del release

In [ ]:
length = df['ImposedSentenceAllChargeInContactEvent']
type = df['SentenceTypeAllChargesAtConvictionInContactEvent']

# print( length.unique()  , '\n') # Some values are ' ', denoting missing
length = pd.to_numeric(length,errors='coerce') # Coerce to numeric
length_NA = length.isnull() # Create a missing dummy
print( np.sum(length_NA),'\n') # 9k missing values of 23k, not so good

print( pd.crosstab(length_NA, type), '\n') # Category 4 is cases where the charges were dismissed

length = length.mask( type == 4, 0) # Replace length with 0 when type ==4
length = length.mask( type == 9, np.nan) # Replace length with np.nan when type == 9

length_NA = length.isnull() # Create a new missing dummy
print( pd.crosstab(length_NA, type), '\n')
print( np.sum(length_NA),'\n') # 274 missing, much better

df['ImposedSentenceAllChargeInContactEvent'] = length # Replace data with cleaned version
del length, type

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?

1. Race is collected on a check all that apply basis.
2. This data is collected because data is important for policy.
3. The census does a good job because it allows participants to check all that apply.
4. The questions are to broad and dont allow for nuanced answers.
5. To much cleaning could lead to nuance being traded off for simplicity and efficiency, when nuance is what allows researchers to find trends.
6. That data could end being used by the wrong people and could potentially negatively affect the lives of reponders.
